<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/dev_branch_EDA_Prediction/Code/ReHealthCo_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReHealthCo Prediction-Prozess

## Vorbereitung 

###CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books (Ziel Reproduzierbarkeit)

In [ ]:
# Löschen aller Variablen
%reset -f

## Importieren der notwendigen Libraries: sqlite3, csv, pandas

In [ ]:
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
from pandas.util import hash_pandas_object
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor

# import regression model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor

# create an object
linreg = LinearRegression()
ElasticNet = ElasticNet()
SGDRegressor = SGDRegressor()
RF = RandomForestRegressor()

### Erstellung einer virtuellen Kopie des GitHub Repositorium DMA2023TeamD in Colab

In [ ]:
!git clone https://github.com/Fuenfgeld/DMA2023TeamD.git

In [ ]:
# Wechsel in den Unterordner csv_data des Github-Projekt-Ordners
%cd /content/DMA2023TeamD/csv_data

In [ ]:
# Auflistung aller Ordnerinhalte
!ls

### Benennung des rhc-DataFrames und Einlesen des entsprechenden anonymisierten csv-Files von Github

In [ ]:
# Pfad-Abkürzung zum Speicherort der anonymisierten csv-Tabelle
csv_path_anonymrhc = "/content/DMA2023TeamD/csv_data"

In [ ]:
# Einlesen der anonymisierten Tabelle als Pandas DataFrame
rhc_df = pd.read_csv(f"{csv_path_anonymrhc}/anonym_rhcpat.csv")

In [ ]:
# Anzeige des DF-Inhalts
rhc_df

# Einlesen passender DataFrames (DF) nach Auswertung durch den EDA-Prozess

### Ziel-DF rhc_all_df mit allen geplanten Vergleichen ohne Bereinigungen

In [ ]:
diag_anz_df = rhc_df[["RHC_PSEUDONYM", "DESC_CON"]].groupby(["RHC_PSEUDONYM"]).count().sort_values("DESC_CON", ascending=False) # Erstellung DF diag_anz_df mit Spalte Diagnosen/Patient als Name
diag_anz_df.rename(columns={"DESC_CON": "DESC_CON_ANZ"}, inplace=True) # Umbenunng der Spalte Diagnosen/Patient in DESC_CON_ANZ
rhc_diagn_df = pd.DataFrame.merge(rhc_df, diag_anz_df, left_on=["RHC_PSEUDONYM"], right_on=["RHC_PSEUDONYM"], how="left") # Spalte Diagnosen/Patient an das gesamte DataFrame hängen
rhc_diagn_df.loc[:, 'DESC_CON_SORT'] = rhc_diagn_df.loc[:, 'DESC_CON_ANZ'] # Spalte DESC_CON_ANZ verdoppeln
rhc_diagn_df['DESC_CON_SORT'].values[rhc_diagn_df['DESC_CON_SORT'] > 2] = 3 # Ersetzen aller Werte bei Patienten > 2 Diagnosen mit einer 3
rhc_diagn_df["HEALTHCARE_EXPENSES_UNCOVERED"] = rhc_diagn_df["HEALTHCARE_EXPENSES_PAT"]-rhc_diagn_df["HEALTHCARE_COVERAGE_PAT"] # neue Spalte HEALTHCARE_EXPENSES uncovered
rhc_diagn_df["VERH_HEALTHCARE_EXPENSES_COV_UNCOV"] = rhc_diagn_df["HEALTHCARE_COVERAGE_PAT"]/rhc_diagn_df["HEALTHCARE_EXPENSES_PAT"] # neue Spalte Verhältnis Ausgaben abgedeckt/nicht abgedeckt mit Spaltennahmen VERH_HEALTHCARE_EXPENSES_COV_UNCOV
rhc_enc_anz_df = rhc_diagn_df[["RHC_PSEUDONYM", "ENCOUNTERCLASS_ENC"]].groupby(["RHC_PSEUDONYM"]).count().sort_values("ENCOUNTERCLASS_ENC", ascending=False) # Anzeige Encounterbegegnungen/Patient, absteigend sortiert
rhc_enc_anz_df.rename(columns={"ENCOUNTERCLASS_ENC": "ENCOUNTERCLASS_ENC_ANZ"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Patient in ENCOUNTERCLASS_ENC_ANZ
rhc_all_df = pd.DataFrame.merge(rhc_diagn_df, rhc_enc_anz_df, left_on=["RHC_PSEUDONYM"], right_on=["RHC_PSEUDONYM"], how="left") # Spalte Encounterbegegnungen/Patient an das gesamte DataFrame hängen
rhc_all_df.loc[:, 'ENCOUNTERCLASS_ENC_ANZ_SORT'] = rhc_all_df.loc[:, 'ENCOUNTERCLASS_ENC_ANZ'] # Spalte ENCOUNTERCLASS_ENC_ANZ verdoppeln
rhc_all_df['ENCOUNTERCLASS_ENC_ANZ_SORT'].values[rhc_all_df['ENCOUNTERCLASS_ENC_ANZ_SORT'] > 2] = 3 # Ersetzen aller Werte bei Patienten > 2 Encounterbegegnungen mit einer 3
rhc_all_df # Anzeige des Ergebnis

### Ziel-DF rhc_all_nowell_df mit allen geplanten Vergleichen ohne wellness bei Encounterbegegnungen

In [ ]:
rhc_all_nowell_df = rhc_all_df.drop(rhc_all_df[ (rhc_all_df['ENCOUNTERCLASS_ENC'] == "wellness")].index) # Löschen der Encounterbegegnung wellness 
# rhc_all_nowell_df.groupby(["ENCOUNTERCLASS_ENC"]).size() # Anzeige der Übersicht
rhc_all_nowell_df

### Ziel-DF groupbypat_df zusammengefasst auf einzelene Patienten mit allen geplanten Vergleichen ohne Bereinigungen

In [ ]:
# Gruppierung nach Patienten, diese sonst doppelt gezählt werden
groupbypat_df = rhc_all_nowell_df[['RHC_PSEUDONYM', 'GENDER_PAT', 'HEALTHCARE_EXPENSES_PAT', 'HEALTHCARE_COVERAGE_PAT', 'DATENBANKART', 'LEBENDIG', 'AGE', 'CODE_CON', 'DESC_CON', 'CONDITIONDAUER', 'TOTALCOST_MED', 'ENCOUNTERCLASS_ENC', 'BEHANDLUNGSDAUER', 'DESC_CON_ANZ', 'DESC_CON_SORT', 'ENCOUNTERCLASS_ENC_ANZ']].groupby(["RHC_PSEUDONYM"]).mean().reset_index()
groupbypat_df

### Ziel-DF anz_pat_dign_enc_df mit Anzeige Patienten/Diagnose sowie Patienten/Encountbegegnung, absteigend sortiert nach Patienten/Diagnose

In [ ]:
pat_anz_nowellrechnung_df = rhc_df.drop(rhc_df[ (rhc_df['ENCOUNTERCLASS_ENC'] == "wellness") ].index) # Löschen der Encounterbegegnung wellness 
diagn_encnowell_df = pat_anz_nowellrechnung_df[["RHC_PSEUDONYM", "DESC_CON", "ENCOUNTERCLASS_ENC"]].groupby(["DESC_CON"]).count().sort_values("RHC_PSEUDONYM", ascending=False) # Benennung des DF
diagn_encnowell_df.rename(columns={"ENCOUNTERCLASS_ENC": "ENCOUNTERCLASS_NOWELL"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Diagnose in ENCOUNTERCLASS_NOWELL
anz_pat_dign_enc_df = diagn_encnowell_df.reset_index()
anz_pat_dign_enc_df.rename(columns={"RHC_PSEUDONYM": "PAT_ANZ", "DESC_CON": "DIAGNOSEN", "ENCOUNTERCLASS_NOWELL": "ENC_BEGEG_NOWELL"}, inplace=True) # Umbenunng der Spalte Encounterbegegnungen/Diagnose in ENCOUNTERCLASS_NOWELL
anz_pat_dign_enc_df # Anzeige DF

### Ziel-DF ohne Extremwerte basierend auf dem DF rhc_all_nowell_df mit allen geplanten Vergleichen ohne wellness bei Encounterbegegnungen

In [ ]:
# Extremwerte herausnehmen
rhc_rein01_df = rhc_all_nowell_df[rhc_all_nowell_df["HEALTHCARE_EXPENSES_PAT"]<1500000]
rhc_rein02_df = rhc_rein01_df[rhc_rein01_df["AGE"]<100]
rhc_rein03_df = rhc_rein02_df[rhc_rein02_df["HEALTHCARE_COVERAGE_PAT"]<20000]
rhc_rein04_df = rhc_rein03_df[rhc_rein03_df["CONDITIONDAUER"]<5000]
rhc_rein05_df = rhc_rein04_df[rhc_rein04_df["TOTALCOST_MED"]<175000]
rhc_rein06_df = rhc_rein05_df[rhc_rein05_df["BEHANDLUNGSDAUER"]<350]
rhc_rein_df = rhc_rein06_df[rhc_rein06_df["DESC_CON_ANZ"]<50]
rhc_rein_df

### folgende essentielle DF wurden erstellt: 
rhc_all_df  =  alles

rhc_all_nowell_df  =  alles ohne wellness

groupbypat_df  =  gemittelt auf einen Patienten

anz_pat_dign_enc_df  =  Patienten/Diagnose sowie Patienten/Encountbegegnung

rhc_rein_df  =  kein wellness keine Extremwerte

# Machinelearning Modelle

In [ ]:
# ACHTUNG nochmal ansehen!!!

# import python packages for ML models

# from sklearn.model_selection import  GridSearchCV

In [ ]:
# Pivot-Table - wahrscheinlich löschen
# dfpred = rhcpatage_df.pivot_table(values="HEALTHCARE_EXPENSES", index=None, columns=["AGE", "GENDER"], aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)
# dfpred.head(20)
# # dfpred.shape

In [ ]:
# Anzeige der Kopfspalten des DF
print(rhc_all_df.columns.tolist())

In [ ]:
dfpred = rhc_all_df[["AGE", "GENDER_PAT", "HEALTHCARE_EXPENSES_PAT"]].groupby(["AGE", "GENDER_PAT"]).mean().reset_index()
dfpred

In [ ]:
# Mischen der Daten
dfpred= dfpred.sample(frac=1)

In [ ]:
# Splitten der Daten in train und test Dataframes
train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

In [ ]:
# Ausgabe der Shapes von train und test Dataframes
dfpred_train, dfpred_test = train_test_split(dfpred, test_size=test_ratio, random_state=0)
dfpred_train.shape, dfpred_test.shape

In [ ]:
print(dfpred_train)

In [ ]:
# Bestimmen der Features und des Zielwertes
X = dfpred_train["AGE"]
y = dfpred_train["HEALTHCARE_EXPENSES_PAT"]

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
y.head()

In [ ]:
y.shape

In [ ]:
pred_test_df = pd.concat([X, y], axis=1)
pred_test_df.shape

In [ ]:
# shuffle
pred_test_df.sample(frac=1)

In [ ]:
X = pred_test_df.drop("HEALTHCARE_EXPENSES_PAT", axis=1)
y = pred_test_df["HEALTHCARE_EXPENSES_PAT"]

In [ ]:
# split data to testing and training dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # ACHTUNG 42!!
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
RF = RandomForestRegressor()

In [ ]:
RF.fit(X_train, y_train)

In [ ]:
ElasticNet.fit(X_train, y_train)

In [ ]:
test_pred_elasticnet = ElasticNet.predict(X_test)
test_pred_elasticnet

In [ ]:
SGDRegressor.fit(X_train, y_train)

In [ ]:
test_pred_SGD = SGDRegressor.predict(X_test)
test_pred_SGD

In [ ]:
# train the model
linreg.fit(X_train, y_train)

In [ ]:
neutest_pred = linreg.predict(pd.DataFrame({"AGE":[40]}))
neutest_pred

In [ ]:
# Making prediction using testing set
test_pred = linreg.predict(X_test)
test_pred

In [ ]:
linreg.score(X_test, y_test) #x groß da Dataframe/Matrix und y klein weil Liste/Vektor

In [ ]:
ElasticNet.score(X_test, y_test) 

In [ ]:
SGDRegressor.score(X_test, y_test) 

In [ ]:
RF.score(X_test, y_test) 

# Version Check

In [ ]:
#python Version
import sys
sys.version_info

In [ ]:
pd.__version__

In [ ]:
csv.__version__

In [ ]:
sns.__version__

In [ ]:
# Überprüfen aller Variablen
# %pip freeze

# Calculate the Checksum of the Data Set

In [ ]:
rhc_df_hashes = hash_pandas_object(rhc_df)
diag_anz_df_hashes = hash_pandas_object(diag_anz_df)
rhc_diagn_df_hashes = hash_pandas_object(rhc_diagn_df)
rhc_enc_anz_df_hashes = hash_pandas_object(rhc_enc_anz_df)
rhc_all_df_hashes = hash_pandas_object(rhc_all_df)
rhc_all_nowell_df_hashes = hash_pandas_object(rhc_all_nowell_df)
groupbypat_df_hashes = hash_pandas_object(groupbypat_df)
pat_anz_nowellrechnung_df_hashes = hash_pandas_object(pat_anz_nowellrechnung_df)
diagn_encnowell_df_hashes = hash_pandas_object(diagn_encnowell_df)
anz_pat_dign_enc_df_hashes = hash_pandas_object(anz_pat_dign_enc_df)
rhc_rein01_df_hashes = hash_pandas_object(rhc_rein01_df)    
rhc_rein02_df_hashes = hash_pandas_object(rhc_rein02_df)
rhc_rein03_df_hashes = hash_pandas_object(rhc_rein03_df)
rhc_rein04_df_hashes = hash_pandas_object(rhc_rein04_df)
rhc_rein05_df_hashes = hash_pandas_object(rhc_rein05_df)
rhc_rein06_df_hashes = hash_pandas_object(rhc_rein06_df)
rhc_rein_df_hashes = hash_pandas_object(rhc_rein_df)

In [ ]:
Liste_rhcpredall_hashes = [rhc_df_hashes, diag_anz_df_hashes, rhc_diagn_df_hashes, rhc_enc_anz_df_hashes, rhc_all_df_hashes, rhc_all_nowell_df_hashes, groupbypat_df_hashes, pat_anz_nowellrechnung_df_hashes, diagn_encnowell_df_hashes, anz_pat_dign_enc_df_hashes, rhc_rein01_df_hashes, rhc_rein02_df_hashes, rhc_rein03_df_hashes, rhc_rein04_df_hashes, rhc_rein05_df_hashes, rhc_rein06_df_hashes, rhc_rein_df_hashes]

In [ ]:
print(Liste_rhcpredall_hashes)

## Value should be 
8997601563088961698

-3795146034381248713

2650178518257614002

-5022892886560696056

8245114429953816038

-5141405102485937931

-8472048597029283422

-840543143014388252

1693621233367357631

-1250212140467252653

9035061648931252140

3979407772490755112

3979407772490755112

-1626434201662496073

-6425855923958030484

3181589982608435791

-7702891464465174751

In [ ]:
print(rhc_df_hashes.sum())
print(diag_anz_df_hashes.sum())
print(rhc_diagn_df_hashes.sum())
print(rhc_enc_anz_df_hashes.sum())
print(rhc_all_df_hashes.sum())
print(rhc_all_nowell_df_hashes.sum())
print(groupbypat_df_hashes.sum())
print(pat_anz_nowellrechnung_df_hashes.sum())
print(diagn_encnowell_df_hashes.sum())
print(anz_pat_dign_enc_df_hashes.sum())
print(rhc_rein01_df_hashes.sum())
print(rhc_rein02_df_hashes.sum())
print(rhc_rein03_df_hashes.sum())
print(rhc_rein04_df_hashes.sum())
print(rhc_rein05_df_hashes.sum())
print(rhc_rein06_df_hashes.sum())
print(rhc_rein_df_hashes.sum())